# Titanic - Machine Learning from Disaster

This is my first attempt at a Kaggle competition. Let us start by importing the necessary libraries: 


In [71]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import re

from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


## Loading and exploring data

### Loading data

In [72]:
df_train = pd.read_csv('train.csv')

display(df_train.head())
print(df_train.dtypes)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


We check for unique values in the 'Sex' column, replace 'male' and 'female' with 0 and 1, respectively, remove letters from the 'Ticket' column keeping only the last numerical part, and convert it to a numeric type column. We also convert the 'Embarked' column to a numeric type column.

In [73]:
#Checking for missing values
print(df_train["Sex"].unique())
#Replacing 
df_train["Sex"] = df_train["Sex"].map({"male": 0, "female": 1})


#Deleting letters from Ticket
df_train['Ticket'] = df_train['Ticket'].str.split().str[-1]
#Converting to numeric
df_train['Ticket'] = pd.to_numeric(df_train['Ticket'], errors='coerce')

#Converting to numeric
df_train['Embarked'] = df_train['Embarked'].map({"S": 0, "C": 1, "Q": 2, "nan": 3})

['male' 'female']


Let's extract and separate the first letters and numbers of the Cabine column into two new columns:

In [74]:
def split_cabin(cabin):
    if not isinstance(cabin, str):
        return [None, None]

    matches = re.findall(r'([A-Za-z]+)(\d+)', cabin)
    if not matches:
        return [None, None]

    letter_number_pairs = [''.join(match) for match in matches]
    most_common_pair = Counter(letter_number_pairs).most_common(1)[0][0]

    most_common_letter, most_common_number = re.match(r'([A-Za-z]+)(\d+)', most_common_pair).groups()

    most_common_number = most_common_number

    return [most_common_letter, most_common_number]

df_train[['Letter_Cabine', 'Numbers_Cabine']] = df_train['Cabin'].apply(lambda x: split_cabin(x)).tolist()
df_train.drop(['Cabin'], axis=1, inplace=True)

df_train["Letter_Cabine"] = df_train["Letter_Cabine"].map({
    "A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G":6, "T":7, "None":8})
df_train["Numbers_Cabine"] = pd.to_numeric(df_train["Numbers_Cabine"], errors='coerce')

display(df_train.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Letter_Cabine,Numbers_Cabine
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,21171.0,7.2500,0.0,NaN,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,17599.0,71.2833,1.0,2.0,85.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,3101282.0,7.9250,0.0,NaN,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803.0,53.1000,0.0,2.0,123.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450.0,8.0500,0.0,NaN,NaN


### Visualizing  data

After loading the data, we will visualize the data to get a better understanding.

In [75]:
print(f"The DataFrame df_train has {df_train.shape[0]} rows and {df_train.shape[1]} columns.\n")

print("Descriptive statistics for the df_train DataFrame, rounded to two decimal places:") 
display(df_train.describe().round(1))

print("\nPercentage of missing values in each column of the df_train DataFrame:")
display(df_train.isnull().mean().mul(100).round(1).reset_index().rename(columns={0: 'Percentage', 'index': 'Column'}))

The DataFrame df_train has 891 rows and 13 columns.

Descriptive statistics for the df_train DataFrame, rounded to two decimal places:


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Letter_Cabine,Numbers_Cabine
count,891.0,891.0,891.0,891.0,714.0,891.0,891.0,887.0,891.0,889.0,200.0,200.0
mean,446.0,0.4,2.3,0.4,29.7,0.5,0.4,298328.4,32.2,0.4,2.4,50.5
std,257.4,0.5,0.8,0.5,14.5,1.1,0.8,657614.4,49.7,0.6,1.5,35.4
min,1.0,0.0,1.0,0.0,0.4,0.0,0.0,3.0,0.0,0.0,0.0,2.0
25%,223.5,0.0,2.0,0.0,20.1,0.0,0.0,14879.0,7.9,0.0,1.0,22.0
50%,446.0,0.0,3.0,0.0,28.0,0.0,0.0,112277.0,14.5,0.0,2.0,43.0
75%,668.5,1.0,3.0,1.0,38.0,1.0,0.0,347082.0,31.0,1.0,3.0,77.2
max,891.0,1.0,3.0,1.0,80.0,8.0,6.0,3101317.0,512.3,2.0,6.0,148.0



Percentage of missing values in each column of the df_train DataFrame:


,Column,Percentage
0,PassengerId,0.0
1,Survived,0.0
2,Pclass,0.0
3,Name,0.0
4,Sex,0.0
5,Age,19.9
6,SibSp,0.0
7,Parch,0.0
8,Ticket,0.4
9,Fare,0.0


### Replace missing values

We will fix the missing values in Age, Cabin and Embarked with 3 different methods.
- replace with mean
- replace with median
- replace with mode, i.e. the most frequent value

In [78]:
df_train_mean = df_train.copy()
df_train_mean['Age'].fillna(df_train_mean['Age'].mean(), inplace=True)
df_train_mean['Ticket'].fillna(df_train_mean['Ticket'].mean(), inplace=True)
df_train_mean['Numbers_Cabine'].fillna(df_train_mean['Numbers_Cabine'].mean(), inplace=True)
df_train_mean['Letter_Cabine'].fillna(df_train_mean['Numbers_Cabine'].mean(), inplace=True)
df_train_mean['Embarked'].fillna(df_train_mean['Embarked'].mean(), inplace=True)


df_train_med = df_train.copy()
df_train_med['Age'].fillna(df_train['Age'].median(), inplace=True)
df_train_med['Ticket'].fillna(df_train_med['Ticket'].median(), inplace=True)
df_train_med['Numbers_Cabine'].fillna(df_train_med['Numbers_Cabine'].median(), inplace=True)
df_train_med['Letter_Cabine'].fillna(df_train_med['Numbers_Cabine'].median(), inplace=True)
df_train_med['Embarked'].fillna(df_train_mean['Embarked'].median(), inplace=True)

#Checking for missing values
print("\nPercentage of missing values in each column of the df_train_mean DataFrame:")
display(df_train_mean.isnull().mean().mul(100).round(1).reset_index().rename(
    columns={0: 'Percentage', 'index': 'Column'}))


Percentage of missing values in each column of the df_train_mean DataFrame:


,Column,Percentage
0,PassengerId,0.0
1,Survived,0.0
2,Pclass,0.0
3,Name,0.0
4,Sex,0.0
5,Age,0.0
6,SibSp,0.0
7,Parch,0.0
8,Ticket,0.0
9,Fare,0.0


Spliting the data into train and test sets:

In [80]:
X = df_train_mean.drop(['Survived', "Name"] , axis=1)
y = df_train_mean['Survived']
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X, y, test_size=0.2, random_state=2)

X = df_train_med.drop(['Survived', "Name"] , axis=1)
y = df_train_med['Survived']
X_train_med, X_test_med, y_train_med, y_test_med = train_test_split(X, y, test_size=0.2, random_state=2)

X = df_train_no_split.drop(['Survived'] , axis=1)
y = df_train_no_split['Survived']
X_train_no_split, X_test_no_split, y_train_no_split, y_test_no_split = train_test_split(
    X, y, test_size=0.2, random_state=2)

## First model - Logistic Regression

### Training

In [51]:
model_mean = LogisticRegression(max_iter=1000)
model_mean.fit(X_train_mean, y_train_mean)

model_med = LogisticRegression(max_iter=1000)
model_med.fit(X_train_med, y_train_med)

LogisticRegression(max_iter=1000)

### Predicting and evaluating

In [52]:
y_pred = model_mean.predict(X_test_mean)
print("Accuracy of Logistic Regression with mean imputation: ", accuracy_score(y_test_mean, y_pred))

y_pred = model_med.predict(X_test_med)
print("Accuracy of Logistic Regression with median imputation: ", accuracy_score(y_test_med, y_pred))

Accuracy of Logistic Regression with mean imputation:  0.6927374301675978
Accuracy of Logistic Regression with median imputation:  0.6759776536312849


## Second model - Random Forest

In [56]:
model_random_forest = RandomForestClassifier(random_state=2)

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [1, 2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5]
}


### Training, predicting and evaluating

#### Model Mean

In [57]:
grid_search = GridSearchCV(estimator=model_random_forest, param_grid=param_grid, cv=5, verbose=2)
grid_search.fit(X_train_mean, y_train_mean)

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=300; total time=   0.3s
[CV] END ..max_depth=1, min_samples_leaf=1, n_

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2),
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'n_estimators': [100, 200, 300, 400, 500]},
             verbose=2)

In [58]:
y_pred = grid_search.predict(X_test_mean)
accuracy_score(y_test_mean, y_pred)

0.7932960893854749

#### Model Median

In [81]:
grid_search = GridSearchCV(estimator=model_random_forest, param_grid=param_grid, cv=5, verbose=2)
grid_search.fit(X_train_med, y_train_med)

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=200; total time=   0.2s
[CV] END ..max_depth=1, min_samples_leaf=1, n_estimators=300; total time=   0.4s
[CV] END ..max_depth=1, min_samples_leaf=1, n_

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2),
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'n_estimators': [100, 200, 300, 400, 500]},
             verbose=2)

In [82]:
y_pred = grid_search.predict(X_test_mean)
accuracy_score(y_test_mean, y_pred)

0.7821229050279329

## Third model - LightGBM

In [84]:
X = df_train_mean.drop(['Survived', "Name"] , axis=1)
y = df_train_mean['Survived']

# Diviser les données en ensembles d'entraînement et de test
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X, y, test_size=0.2, random_state=2)

# Créer un jeu de données pour lightgbm
lgb_train = lgb.Dataset(X_train_mean, y_train_mean)
lgb_eval = lgb.Dataset(X_test_mean, y_test_mean, reference=lgb_train)

# Spécifier les paramètres de configuration
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 100,
    'learning_rate': 0.12,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Entraîner le modèle
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval)

# Prédire les classes pour l'ensemble de test
y_pred = gbm.predict(X_test_mean)

# Convertir les prédictions en classes binaires
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

# Calculer la précision
accuracy = accuracy_score(y_test_mean, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: Name: object

## Fourth model - XGBoost

In [62]:
# Définir les données
X = df_train_mean.drop(['Survived', "Name"] , axis=1)
y = df_train_mean['Survived']

# Diviser les données en ensembles d'entraînement et de test
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X, y, test_size=0.2, random_state=2)

# Créer un jeu de données pour xgboost
dtrain = xgb.DMatrix(X_train_mean, label=y_train_mean)
dtest = xgb.DMatrix(X_test_mean, label=y_test_mean)

# Spécifier les paramètres de configuration
params = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.1,
    'silent': 1,
    'n_estimators': 100
}

# Entraîner le modèle
bst = xgb.train(params, dtrain, num_boost_round=20)

# Prédire les classes pour l'ensemble de test
y_pred = bst.predict(dtest)

# Convertir les prédictions en classes binaires
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

# Calculer la précision
accuracy = accuracy_score(y_test_mean, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.01%


/usr/local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [02:45:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimators", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [63]:
# Définir les données
X = df_train_mean.drop(['Survived', "Name"], axis=1)
y = df_train_mean['Survived']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Normaliser les données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Créer un modèle avec régularisation, dropout et batch normalization
model = tf.keras.models.Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compiler le modèle avec un optimiseur ajusté
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle avec une callback pour l'arrêt précoce
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train, epochs=200, batch_size=16, validation_split=0.1, callbacks=[callback])

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Epoch 1/200
40/40 [==============================] - 1s 7ms/step - loss: 1.1170 - accuracy: 0.6078 - val_loss: 0.8896 - val_accuracy: 0.7917
Epoch 2/200
40/40 [==============================] - 0s 4ms/step - loss: 1.0375 - accuracy: 0.6547 - val_loss: 0.8432 - val_accuracy: 0.7917
Epoch 3/200
40/40 [==============================] - 0s 3ms/step - loss: 0.9462 - accuracy: 0.7063 - val_loss: 0.8070 - val_accuracy: 0.8056
Epoch 4/200
40/40 [==============================] - 0s 3ms/step - loss: 0.9623 - accuracy: 0.7063 - val_loss: 0.7742 - val_accuracy: 0.8194
Epoch 5/200
40/40 [==============================] - 0s 4ms/step - loss: 0.8600 - accuracy: 0.7531 - val_loss: 0.7453 - val_accuracy: 0.8333
Epoch 6/200
40/40 [==============================] - 0s 3ms/step - loss: 0.8541 - accuracy: 0.7500 - val_loss: 0.7304 - val_accuracy: 0.8056
Epoch 7/200
40/40 [==============================] - 0s 3ms/step - loss: 0.8910 - accuracy: 0.7359 - val_loss: 0.7144 - val_accuracy: 0.8056
Epoch 8/200
4

In [66]:
# Définir les données
X = df_train_med.drop(['Survived', "Name"], axis=1)
y = df_train_med['Survived']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Normaliser les données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Créer un modèle avec régularisation, dropout et batch normalization
model = tf.keras.models.Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compiler le modèle avec un optimiseur ajusté
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle avec une callback pour l'arrêt précoce
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.fit(X_train, y_train, epochs=200, batch_size=16, validation_split=0.1, callbacks=[callback])

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Epoch 1/200
40/40 [==============================] - 1s 7ms/step - loss: 1.2904 - accuracy: 0.4906 - val_loss: 0.9734 - val_accuracy: 0.5000
Epoch 2/200
40/40 [==============================] - 0s 3ms/step - loss: 1.0470 - accuracy: 0.6391 - val_loss: 0.9587 - val_accuracy: 0.5139
Epoch 3/200
40/40 [==============================] - 0s 4ms/step - loss: 0.9631 - accuracy: 0.6687 - val_loss: 0.9467 - val_accuracy: 0.5000
Epoch 4/200
40/40 [==============================] - 0s 3ms/step - loss: 0.9396 - accuracy: 0.6969 - val_loss: 0.9267 - val_accuracy: 0.5556
Epoch 5/200
40/40 [==============================] - 0s 3ms/step - loss: 0.9320 - accuracy: 0.6859 - val_loss: 0.9056 - val_accuracy: 0.5833
Epoch 6/200
40/40 [==============================] - 0s 3ms/step - loss: 0.8989 - accuracy: 0.7219 - val_loss: 0.8891 - val_accuracy: 0.6667
Epoch 7/200
40/40 [==============================] - 0s 3ms/step - loss: 0.8726 - accuracy: 0.7344 - val_loss: 0.8805 - val_accuracy: 0.7083
Epoch 8/200
4